In [1]:
from langchain_openai import ChatOpenAI
from langchain.tools import TavilySearchResults
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.prompts import ChatPromptTemplate
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

# Set up configuration
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
LLM_MODEL = "gpt-4o-mini"
MAX_SEARCH_RESULTS = 3

# Set API keys
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

In [3]:
search_tool = TavilySearchResults(max_results=MAX_SEARCH_RESULTS)

llm = ChatOpenAI(temperature=0, model=LLM_MODEL)
        
system_prompt = """
You are a financial researcher gathering information about companies for investment analysis.

Focus on finding recent and relevant information about the company such as:
1. Quarterly financial results and earnings
2. Analyst ratings and price targets
3. Business developments, product launches, partnerships
4. Market trends affecting the company
5. Competitive landscape

Only consider information relevant for short-term investment decisions (last 3 months).
Make sure to search for the most recent information available.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "Research {ticker} ({company_name}) for short-term investment analysis."),
    ("ai", "{agent_scratchpad}")
])
 

In [4]:
      
agent = create_openai_functions_agent(
    llm=llm,
    tools=[search_tool],
    prompt=prompt
)
agent


RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'company_name', 'ticker'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\nYou are a financial researcher gathering information about companies for investment analysis.\n\nFocus on finding recent and relevant information about the company such as:\n1. Quarterly financial results and earnings\n2. Analyst ratings and price targets\n3. Business developments, product launches, partnerships\n4. Market trends affecting the company\n5. Competitive landscape\n\nOnly consider information relevant for short-term investment decisions (last 3 months).\nMake sure to search for the most recent information available.\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(in

In [5]:

agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True,
    handle_parsing_errors=True
)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'company_name', 'ticker'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\nYou are a financial researcher gathering information about companies for investment analysis.\n\nFocus on finding recent and relevant information about the company such as:\n1. Quarterly financial results and earnings\n2. Analyst ratings and price targets\n3. Business developments, product launches, partnerships\n4. Market trends affecting the company\n5. Competitive landscape\n\nOnly consider information relevant for short-term investment decisions (last 3 months).\nMake sure to search for the most recent information available.\n'), additional_kwargs

In [8]:
result = agent_executor.invoke({
        "ticker": "AAPL",
        "company_name": "Apple"
    })
result



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'AAPL Apple quarterly financial results earnings analyst ratings price targets business developments product launches partnerships market trends competitive landscape October 2023'}`


[{'title': 'aapl-20230930 - SEC.gov', 'url': 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000106/aapl-20230930.htm', 'content': 'Americas:                 \nNet sales   $   162,560\xa0            $   169,658\xa0            $   153,306\xa0  \nOperating income    $   60,508\xa0         $   62,683\xa0         $   53,382\xa0 \nEurope:               \nNet sales   $   94,294\xa0         $   95,118\xa0         $   89,307\xa0 \nOperating income    $   36,098\xa0         $   35,233\xa0         $   32,505\xa0 \nGreater China:                \nNet sales   $   72,559\xa0         $   74,200\xa0         $   68,366 [...] Gross margin:                 \nProducts    $   108,803\xa0            $   114,728\xa0    

{'ticker': 'AAPL',
 'company_name': 'Apple',
 'output': "Here is the recent information on Apple Inc. (AAPL) relevant for short-term investment analysis:\n\n### 1. Quarterly Financial Results and Earnings\n- **Q4 2023 Results**: \n  - **Net Sales**: $94.93 billion\n  - **Earnings**: $14.74 billion\n  - **Gross Margin**: 44.1% (up from 43.3% in Q3 2023)\n  - **Operating Income**: $30.33 billion\n  - **Regional Performance**:\n    - **Americas**: Net sales of $162.56 billion\n    - **Europe**: Net sales of $94.29 billion\n    - **Greater China**: Net sales of $72.56 billion\n    - **Japan**: Net sales of $24.26 billion\n    - **Rest of Asia Pacific**: Net sales of $29.62 billion\n  - [Source: SEC Filing](https://www.sec.gov/Archives/edgar/data/320193/000032019323000106/aapl-20230930.htm)\n\n### 2. Analyst Ratings and Price Targets\n- **Earnings Per Share Estimate**: $1.61\n- **Analyst Recommendations**:\n  - Strong Buy: Majority of analysts recommend a strong buy.\n  - Buy: Several analy

In [6]:
ticker = "MSFT"
company_name = "Microsoft Corporation"
result = agent_executor.invoke({
        "ticker": ticker,
        "company_name": company_name
    })

print (
    {
        "ticker": ticker,
        "company_name": company_name,
        "search_results": result["output"],
        "raw_results": result
    }
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Microsoft Corporation MSFT recent quarterly results earnings analyst ratings business developments market trends competitive landscape'}`


[{'title': 'Microsoft Corporation (MSFT) Analyst Ratings, Estimates & Forecasts', 'url': 'https://finance.yahoo.com/quote/MSFT/analysis/', 'content': '| Currency in USD | Current Qtr. (Mar 2025) | Next Qtr. (Jun 2025) | Current Year (2025) | Next Year (2026) |\n| --- | --- | --- | --- | --- |\n| No. of Analysts | 32 | 31 | 55 | 54 |\n| Avg. Estimate | 68.46B | 72.68B | 277.27B | 315.96B |\n| Low Estimate | 67.14B | 71.47B | 274.86B | 304.21B |\n| High Estimate | 69.36B | 74.63B | 283.15B | 341.76B |\n| Year Ago Sales | 61.86B | 64.73B | 245.12B | 277.27B |\n| Sales Growth (year/est) | 10.67% | 12.29% | 13.11% | 13.95% |\nEarnings History [...] Analyst Price Targets\n420.00\n506.80 Average\n409.75 Current\n650.00 High\nEarnings Estimate\n| Currency in US